# Fit-File-Parser

```markdown
Autor:          Maik 'Schrottie' Bischoff
Beschreibung:   Parse Garmin .fit files and write the data to a CSV file or database.
Version:        0.2
Stand:          06.04.2023
```
##### ToDo:

<ul style="font-size: 85%;font-family: monospace">
    <li>Function to create a table in the database with the fields from the .fit-file.</li>
    <li>Function for checking whether all fields from the current .fit-file exist in an existing table, if necessary updating the table with the new fields.</li>
</ul>

##### Change-/Versionlog:

<table>
    <tr>
        <td>
            <span style="font-size: 85%;font-family: monospace">0.2:</span>
        </td>
        <td>
            <ul style="font-size: 85%;font-family: monospace">
                <li>Added a function to recursively search a directory for .fit files to allow processing multiple files at the same time.</li>
                <li>Adjusting the field label for the longitude (position_long --> position_lon) so that the field is recognized properly when the data is processed further (e.g. in ArcGIS Pro)</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>
            <span style="font-size: 85%;font-family: monospace">0.1:</span>
        </td>
        <td>
            <ul style="font-size: 85%;font-family: monospace">
                <li>Basic function for processing a .fit file.</li>
                <li>Converting the crude Garmin coordinate format (semicircles) into 'real' coordinates.</li>
            </ul>
        </td>
    </tr>
</table>

In [40]:
import os
import fitparse
import pandas as pd
import datetime
from sqlalchemy import create_engine

# Set global variables
today = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
fit_path = 'U:\\pyScripte\\fit_file_parser\\testdaten'
csv_path = 'U:\\pyScripte\\fit_file_parser\\testdaten'

In [41]:
# Function for converting Garmin semicircles to degrees.
def semicircles_to_degree(semicircles):
    
    return semicircles * (180 / 2 ** 31)


In [42]:
# Function to check if a file has a .fit extension
def is_fit_file(filename):
    return filename.lower().endswith('.fit')

In [43]:
# Function to read the data from a single .fit file
def read_fit_file(file_path):
    fitfile = fitparse.FitFile(file_path)

    data = []
    for record in fitfile.get_messages('record'):
        # Create an empty dictionary to hold the data for this record
        record_data = {}
        for data_point in record:
            # Convert Garmin-Semicircles to Degree
            if data_point.name == 'position_lat' or data_point.name == 'position_long':
                record_data[data_point.name] = semicircles_to_degree(data_point.value)
            else:
                record_data[data_point.name] = data_point.value
        data.append(record_data)

    # Convert the list of data to a Pandas DataFrame
    df = pd.DataFrame(data)
    # Give the longitude a proper field name
    df = df.rename(columns={'position_long': 'position_lon'})

    return df

In [44]:
# Function to recursively find all .fit files in a directory
def find_fit_files(directory):
    fit_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if is_fit_file(file):
                fit_files.append(os.path.join(root, file))
    return fit_files


In [45]:
# Function to do all the funny things
def extract_fit_data_to_df(fit_path, csv_path):
    # Set the directory to search for .fit files
    if fit_path:
        directory = fit_path
    else:
        directory = os.getcwd()

    # Find all .fit files in the directory
    fit_files = find_fit_files(directory)

    # Read the data from each .fit file and combine it into a single DataFrame
    dfs = []
    for fit_file in fit_files:
        df = read_fit_file(fit_file)
        dfs.append(df)
    combined_df = pd.concat(dfs)

    # Write the combined DataFrame to a CSV file
    if csv_path:
        combined_df.to_csv(f'{csv_path}/{today}_output.csv', index=False)
    else:
        combined_df.to_csv('output.csv', index=False)

In [ ]:
def load_dotenv():

    server=os.getenv('DB_HOST')
    database=os.getenv('DB_NAME')
    username=os.getenv('DB_USER')
    password=os.getenv('USER_PASSWORD')
    
    return server, database, username, password

In [ ]:
# Function to write pandas DataFrame to database using sqlalchemy.
def write_to_mssql(db_type="PGSQL", df, table_name):
   
    # Load environment variables from .env file
    server, database, username, password = load_dotenv()

    # Create sqlalchemy engine
    if db_type == 'PGSQL':
        engine = create_engine(f'postgresql://{username}:{password}@{server}/{database}')
    elif db_type  == 'MSSQL':
        engine = create_engine(f'mssql+pymssql://{username}:{password}@{server}/{database}')
    elif db_type == 'MARIADB':
        engine = create_engine(f'mysql+pymysql://{username}:{password}@{server}/{database}')
    elif db_type == 'ORACLE':
        engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{server}/{database}')
    elif db_type == 'MYSQL':
        engine = create_engine(f'mysql+pymysql://{username}:{password}@{server}/{database}')
    elif db_type == 'SQLITE':
        engine = create_engine('sqlite:///mydatabase.db')
    else:
        return

    # Write DataFrame to database
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

    print(f"DataFrame successfully written to table {table_name} in MS SQL database.")


In [46]:
extract_fit_data_to_df(fit_path, csv_path)